In [ ]:
! echo "::group::Install Dependencies"
%pip install uv
! uv pip install "git+https://github.com/ibm-granite-community/utils.git" \
    langgraph \
    langchain \
    langchain_ibm
! echo "::endgroup::"


In [ ]:
from ibm_granite_community.notebook_utils import get_env_var
from langchain_core.utils.utils import convert_to_secret_str
from langchain.chat_models import init_chat_model

model = "ibm/granite-3-3-8b-instruct"

model_parameters = {
    "temperature": 0,
    "max_completion_tokens": 500,
    "repetition_penalty": 1.05,
}

llm = init_chat_model(
    model=model,
    model_provider="ibm",
    url=convert_to_secret_str(get_env_var("WATSONX_URL")),
    apikey=convert_to_secret_str(get_env_var("WATSONX_APIKEY")),
    project_id=get_env_var("WATSONX_PROJECT_ID"),
    params=model_parameters,
)

In [ ]:
#definte mock tools for now as test
from langchain.tools import tool

@tool
def get_weather(destination: str, month: str) -> str:
    """Get weather for a destination and month."""
    return f"The weather in {destination} in {month} is cool and dry, highs ~15°C."

@tool
def get_flight_info(destination: str, month: str) -> str:
    """Get flight cost for a destination and month."""
    return f"Flights to {destination} in {month} average around $1200 USD round-trip."

@tool
def get_hotel_prices(destination: str, month: str) -> str:
    """Get hotel prices for a destination and month."""
    return f"Hotels in {destination} range from $100 to $300 per night in {month}."

tools = [get_weather, get_flight_info, get_hotel_prices]


In [ ]:
from langchain_core.prompts import PromptTemplate

template = """You are a helpful travel planning assistant that uses the ReAct (Reasoning + Acting) framework to answer travel-related questions.

1. Tool Usage
You have access to the following tools:
{tools}

2.Response
If the user’s question requires an external tool or related to the domain of any tool listed above, use this response format:

Thought: Explain why the tool is needed.
Collect: Ask for missing parameters, or skip if all are present.
Action: {{"name": "tool_name", "args":{{}}}}
Observation: filled later with the tool result
(This loop may repeat multiple times. When you know the final answer:)

Thought: I now know the final answer
Final Answer: [Your complete travel recommendation]

3. Rules
* Always start with Thought.
* Only use tools listed in "Available Tools", which are the following: {tool_names}

Begin!

Question: {input}
{agent_scratchpad}"""

prompt = PromptTemplate(
    template=template,
    input_variables=["input", "agent_scratchpad", "tools", "tool_names"],
)

In [ ]:
from langchain.agents import create_react_agent, AgentExecutor

agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [ ]:
response = agent_executor.invoke({"input": "I want to visit Tokyo next month, which will be April. What should I expect in terms of weather and cost?"})

In [ ]:
print(response["output"])